
# Homework 1 — Part 3: Allocations (Teacher-matching Final Version)

This notebook follows the instructor's solution exactly:
- Uses sheet **'excess returns'** from `multi_asset_etf_data.xlsx` (monthly excess returns).
- Annualization factor = **12** (monthly → annual).
- Portfolios constructed:
  - Equally-weighted (EW): w_i = 1/n
  - Risk-parity (RP): w_i ∝ 1/σ_i^2 (use full-sample variances)
  - Regularized (REG): w ∝ Σ̂^{-1} μ  with Σ̂ = (Σ + Σ_D)/2
  - Tangency (TAN): w ∝ Σ^{-1} μ
- **All portfolios are rescaled** so that their *annualized mean* = **μ_target = 0.12** (instructor's target).
- Performance metrics reported: **Annualized Mean, Annualized Vol, Annualized Sharpe** (rf already removed in excess returns).


In [2]:

import pandas as pd, numpy as np
from pathlib import Path
from numpy.linalg import inv

DATA_PATH = Path('multi_asset_etf_data.xlsx')
df = pd.read_excel(DATA_PATH, sheet_name='excess returns')
# try set date index
try:
    df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])
    df = df.set_index(df.columns[0])
except Exception:
    pass

returns = df.select_dtypes(include=[np.number]).dropna(how='all').sort_index()
ANN = 12  # monthly data -> annualize by 12

mu_month = returns.mean()
cov_month = returns.cov()
mu_ann = mu_month * ANN
cov_ann = cov_month * ANN

assets = returns.columns.tolist()
n = len(assets)
print('Assets:', assets)
print('Observations:', len(returns))

Assets: ['BWX', 'DBC', 'EEM', 'EFA', 'HYG', 'IEF', 'IYR', 'PSP', 'QAI', 'SPY', 'TIP']
Observations: 164


In [7]:

# Build raw/normalized weights per definitions
ones = np.ones(n)
w_ew = np.ones(n) / n

# RP: proportional to 1/variance
vars = np.diag(cov_ann.values)
w_rp_raw = 1.0 / vars
w_rp_unit = w_rp_raw / np.sum(w_rp_raw)

# REG: Sigma_hat = 0.5*(Sigma + Sigma_D)
Sigma = cov_ann.values
Sigma_D = np.diag(np.diag(Sigma))
Sigma_hat = 0.5 * (Sigma + Sigma_D)
w_reg_raw = np.linalg.inv(Sigma_hat).dot(mu_ann.values)
w_reg_unit = w_reg_raw / (ones.dot(w_reg_raw))

# Tangency (unconstrained, unit direction)
w_tan_raw = np.linalg.inv(cov_ann.values).dot(mu_ann.values)
w_tan_unit = w_tan_raw / (ones.dot(w_tan_raw))

# Target mean (instructor-specified)
mu_target = 0.12

def rescale_to_target(w, mu_ann_vals, target):
    cur = w.dot(mu_ann_vals)
    if np.isclose(cur, 0):
        return w
    return w * (target / cur)

w_ew_scaled = rescale_to_target(w_ew, mu_ann.values, mu_target)
w_rp_scaled = rescale_to_target(w_rp_unit, mu_ann.values, mu_target)
w_reg_scaled = rescale_to_target(w_reg_unit, mu_ann.values, mu_target)
w_tan_scaled = rescale_to_target(w_tan_unit, mu_ann.values, mu_target)

# compute performance from returns time series
def stats_from_returns(w, returns_df, ann=12):
    port = returns_df.dot(w)
    mean_ann = port.mean() * ann
    vol_ann = port.std(ddof=1) * np.sqrt(ann)
    sharpe_ann = mean_ann / vol_ann if vol_ann > 0 else np.nan
    return mean_ann, vol_ann, sharpe_ann

portfolios = {
    'Equal Weights': w_ew_scaled,
    'Risk Parity': w_rp_scaled,
    'Regularized Portfolio': w_reg_scaled,
    'Tangency Portfolio': w_tan_scaled
}

rows = []
for name, w in portfolios.items():
    mean_ann, vol_ann, sharpe_ann = stats_from_returns(w, returns, ann=ANN)
    rows.append({'Portfolio': name, 'Annualized Mean': mean_ann, 'Annualized Vol': vol_ann, 'Annualized Sharpe': sharpe_ann, 'SumWeights': w.sum()})
results = pd.DataFrame(rows).set_index('Portfolio')
weights_df = pd.DataFrame(portfolios, index=assets)

display(results.round(6))
display(weights_df)

,Annualized Mean,Annualized Vol,Annualized Sharpe,SumWeights
Portfolio,,,,
Equal Weights,0.12,0.291527,0.411625,2.987328
Risk Parity,0.12,0.309515,0.387703,5.507806
Regularized Portfolio,0.12,0.102604,1.169541,0.789321
Tangency Portfolio,0.12,0.074765,1.605037,-0.227585


,Equal Weights,Risk Parity,Regularized Portfolio,Tangency Portfolio
BWX,0.271575,0.522879,-0.675374,-0.621364
DBC,0.271575,0.122905,-0.136195,0.025317
EEM,0.271575,0.107717,-0.066165,0.140124
EFA,0.271575,0.150554,0.037302,-0.120777
HYG,0.271575,0.583857,0.316781,0.189482
IEF,0.271575,0.873276,0.393506,1.065661
IYR,0.271575,0.121273,0.080277,-0.199617
PSP,0.271575,0.075284,0.049547,-0.063041
QAI,0.271575,1.452202,-0.156500,-1.643297
SPY,0.271575,0.170397,0.623712,1.116095


In [ ]:

OUT_DIR = Path('/mnt/data')
results.to_csv(OUT_DIR / 'allocations_teachermatch_results.csv')
weights_df.to_csv(OUT_DIR / 'allocations_teachermatch_weights.csv')
print('Saved CSVs to /mnt/data') 
results, weights_df.head(12)